In [ ]:
# Load in our libraries
import pandas as pd
import numpy as np
import re
import sklearn
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

import warnings
warnings.filterwarnings('ignore')

# Going to use these 5 base models for the stacking
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn.svm import SVC
from sklearn.model_selection import KFold

## Em progesso..

In [ ]:
# settings
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt

In [ ]:
df1 = pd.read_csv('../input/credit-card-approval-prediction/application_record.csv')
df2 = pd.read_csv('../input/credit-card-approval-prediction/credit_record.csv')

In [ ]:
df1.head(5)


In [ ]:
df2.head(2)

### limpando os dados

In [ ]:
# Verificando a presença de valores nulos no data frame
df1.isnull().sum().sum()

In [ ]:
# Verificando a presença de valores nulos no data frame
df2.isnull().sum().sum()

In [ ]:
# Verificando a presença de NA's no data frame
df1.isnull().values.any()

In [ ]:
#REmovendo os Null/NA
df1 = df1.dropna()

In [ ]:
#verificando se sobrou algum Null/NA
df1.isnull().values.any()

### Verificando o numero de linhas restantes

In [ ]:
index = df1.index
number_of_rows = len(index)
print(number_of_rows)

In [ ]:
index = df2.index
number_of_rows = len(index)
print(number_of_rows)

### Juntando as duas bases de dados

In [ ]:
df3 = pd.merge(df1, df2, on='ID')
df3.head(3)

In [ ]:
#Verificando a quantidade de dados depois da junção dos conjuntos
index = df3.index
number_of_rows = len(index)
print(number_of_rows)

* É possível notar que a base de dados aumentou, portanto houve duplicatas de valores. Precisamos remove-lás.

In [ ]:
#verificando se sobrou algum Null/NA
df3.isnull().values.any()

In [ ]:
#Tirando os ID duplicados
df3 = df3.drop_duplicates('ID',keep='first')



In [ ]:
#Verificando a quantidade de dados depois da junção dos conjuntos
df3

### Transformando em dummies

In [ ]:
#Removendo a variável CODE_GENDER para não ter viés sexista na base de dados
df3 = df3.drop(columns=['CODE_GENDER'])
df3.head(3)

In [ ]:
# Trasnformando todos de valores Y ou N em dummies, sendo 1 para Y
dummy1 = pd.get_dummies(df3.FLAG_OWN_CAR)
df3['FLAG_OWN_CAR'] = dummy1['Y']

dummy2 = pd.get_dummies(df3.FLAG_OWN_REALTY)
df3['FLAG_OWN_REALTY'] = dummy2['Y']


#Vendo as classes das variáveis categóricas
#print(df3['NAME_INCOME_TYPE'].unique())

#Vendo as classes das variáveis categóricas
#print(df3['NAME_EDUCATION_TYPE'].unique())

#Vendo as classes das variáveis categóricas
#print(df3['OCCUPATION_TYPE'].unique())

#Vendo as classes das variáveis categóricas
#print(df3['STATUS'].unique())




In [ ]:
###criando uma variável ordinal para o nível de escolaridade
#df3['NAME_EDUCATION_TYPE'] =



## Vamos tentar entender a capacidade de pagamento dos individuos e enquadrá-lo em categorias

In [ ]:
#### Vamos ver como são os níveis de consumo por categorias sociais


fig, axes = plt.subplots(1, 4, figsize=(15, 5), sharey=True)
fig.suptitle('Consumo por característica')

# Bulbasaur
sns.barplot(ax=axes[0], x=df3.NAME_INCOME_TYPE, y=df3.AMT_INCOME_TOTAL).tick_params(labelrotation=45)
axes[0].set_title("Ocupação")


# Charmander
sns.barplot(ax=axes[1], x=df3.NAME_EDUCATION_TYPE, y=df3.AMT_INCOME_TOTAL).tick_params(labelrotation=45)
axes[1].set_title("Escolaridade")

# Squirtle
sns.barplot(ax=axes[2], x=df3.NAME_FAMILY_STATUS, y=df3.AMT_INCOME_TOTAL).tick_params(labelrotation=45)
axes[2].set_title("Estatus Civil")

#
sns.barplot(ax=axes[3], x=df3.NAME_HOUSING_TYPE, y=df3.AMT_INCOME_TOTAL).tick_params(labelrotation=45)
axes[3].set_title("Moradia")



In [ ]:

###criando UM catplot individual para a variavel OCCUPATION_TYPE em relação ao poder de compra AMT_INCOME_TOTAL
plt.figure(figsize =(10,5))
ax = sns.barplot(x="OCCUPATION_TYPE", y="AMT_INCOME_TOTAL",data=df3).set_title('Consumo por profissão')
plt.xticks(rotation=60)

### Como há várias categorias, dividiremos todas elas pelo poder de consumo

In [ ]:
#média de consumo por profissão
df4 = df3.groupby(['OCCUPATION_TYPE']).mean().sort_values(['AMT_INCOME_TOTAL'], ascending=False)
df4['AMT_INCOME_TOTAL']

In [ ]:


# Como temos 18 profissões, vamos criar um indice de impacto de 6 níveis, de acordo com o poder de consumo
df3['OCCUPATION_TYPE'] = df3['OCCUPATION_TYPE'].replace(['Managers','Realty agents'],6)
df3['OCCUPATION_TYPE'] = df3['OCCUPATION_TYPE'].replace(['Drivers','Accountants','IT staff','Private service staff'],5)
df3['OCCUPATION_TYPE'] = df3['OCCUPATION_TYPE'].replace(['High skill tech staff','HR staff','Core staff','Laborers'],4)
df3['OCCUPATION_TYPE'] = df3['OCCUPATION_TYPE'].replace(['Security staff','Sales staff','Secretaries','Medicine staff'],3)
df3['OCCUPATION_TYPE'] = df3['OCCUPATION_TYPE'].replace(['Drivers','Accountants','IT staff','Private service staff'],2)
df3['OCCUPATION_TYPE'] = df3['OCCUPATION_TYPE'].replace(['Waiters/barmen staff','Cleaning staff','Cooking staff','Low-skill Laborers'],1)
df3['OCCUPATION_TYPE'] = df3['OCCUPATION_TYPE'].apply(pd.to_numeric)

In [ ]:
# Fazendo o mesmo para educação
df5 = df3.groupby(['NAME_EDUCATION_TYPE']).mean().sort_values(['AMT_INCOME_TOTAL'], ascending=False)
df5['AMT_INCOME_TOTAL']

In [ ]:
df3['NAME_EDUCATION_TYPE'] = df3['NAME_EDUCATION_TYPE'].replace(['Academic degree'],5)
df3['NAME_EDUCATION_TYPE'] = df3['NAME_EDUCATION_TYPE'].replace(['Higher education'],4)
df3['NAME_EDUCATION_TYPE'] = df3['NAME_EDUCATION_TYPE'].replace(['Incomplete higher'],3)
df3['NAME_EDUCATION_TYPE'] = df3['NAME_EDUCATION_TYPE'].replace(['Secondary / secondary special'],2)
df3['NAME_EDUCATION_TYPE'] = df3['NAME_EDUCATION_TYPE'].replace(['Lower secondary'],1)
df3['NAME_EDUCATION_TYPE'] = df3['NAME_EDUCATION_TYPE'].apply(pd.to_numeric)
print(df3['NAME_EDUCATION_TYPE'].unique())

In [ ]:
# Fazendo o mesmo para finalidade de uso do crédito
df6 = df3.groupby(['NAME_INCOME_TYPE']).mean().sort_values(['AMT_INCOME_TOTAL'], ascending=False)
df6['AMT_INCOME_TOTAL']


In [ ]:
df3['NAME_INCOME_TYPE'] = df3['NAME_INCOME_TYPE'].replace(['Pensioner'],5)
df3['NAME_INCOME_TYPE'] = df3['NAME_INCOME_TYPE'].replace(['Commercial associate'],4)
df3['NAME_INCOME_TYPE'] = df3['NAME_INCOME_TYPE'].replace(['State servant'],3)
df3['NAME_INCOME_TYPE'] = df3['NAME_INCOME_TYPE'].replace(['Working'],2)
df3['NAME_INCOME_TYPE'] = df3['NAME_INCOME_TYPE'].replace(['Student'],1)
df3['NAME_INCOME_TYPE'] = df3['NAME_INCOME_TYPE'].apply(pd.to_numeric)
print(df3['NAME_INCOME_TYPE'].unique())

In [ ]:
# Fazendo o mesmo para finalidade de uso do crédito
df7 = df3.groupby(['NAME_HOUSING_TYPE']).mean().sort_values(['AMT_INCOME_TOTAL'], ascending=False)
df7['AMT_INCOME_TOTAL']

In [ ]:
df3['NAME_HOUSING_TYPE'] = df3['NAME_HOUSING_TYPE'].replace(['Office apartment'],6)
df3['NAME_HOUSING_TYPE'] = df3['NAME_HOUSING_TYPE'].replace(['Co-op apartment'],5)
df3['NAME_HOUSING_TYPE'] = df3['NAME_HOUSING_TYPE'].replace(['Rented apartment'],4)
df3['NAME_HOUSING_TYPE'] = df3['NAME_HOUSING_TYPE'].replace(['House / apartment'],3)
df3['NAME_HOUSING_TYPE'] = df3['NAME_HOUSING_TYPE'].replace(['Municipal apartment'],2)
df3['NAME_HOUSING_TYPE'] = df3['NAME_HOUSING_TYPE'].replace(['With parents'],1)
df3['NAME_HOUSING_TYPE'] = df3['NAME_HOUSING_TYPE'].apply(pd.to_numeric)
print(df3['NAME_HOUSING_TYPE'].unique())

In [ ]:
#Como foi visto no gráfico inicial, a variável estado civil não varia muito de consumo de acordo como status,
# portanto vamos remove-lá junto as demais desnecessárias

df3 = df3.drop(columns=['NAME_FAMILY_STATUS'])
df3 = df3.drop(columns=['ID'])
#Vamos tira a variável FLAG_MOBIL , CNT_CHILDREN e FLAG_WORK_PHONE pois também não traz informação relevante
df3 = df3.drop(columns=['FLAG_MOBIL'])
df3 = df3.drop(columns=['FLAG_WORK_PHONE'])
df3 = df3.drop(columns=['CNT_CHILDREN'])
df3 = df3.drop(columns=['FLAG_PHONE'])
df3 = df3.drop(columns=['FLAG_EMAIL'])
df3.head(2)

#### Agora vamos analisar a variavel target

##### Temos duas categorias de indivíduos: com atrasos de pagamentos, e sem atrasos. Portanto vamos categorizá-los como **inadimplentes e adimplentes**. A decisão de quem é adimplente ou inadimplente é relativo e depende dos interesses internos das instituições, mas para simplificação do modelo fazeremos dessa forma.

Você deve estar se perguntando se não seria interessante estimar uma regressão linear antes de transformar a variável target em dummy. A questão é que as variáveis explicativas precisam ter distribuição normal para obter os melhores estimadores de MQO, o que não acontece no nosso conjunto de dados. Portando levaremos a nossa análise a modelos não linear.

In [ ]:
#letras são adimplentes e números inadimplentes
df3['STATUS'] = df3['STATUS'].replace(['C'],0)
df3['STATUS'] = df3['STATUS'].replace(['X'],0)
df3['STATUS'] = df3['STATUS'].apply(pd.to_numeric) 
df3['STATUS'] = np.where(df3['STATUS']<1, 0, 1)
print(df3['STATUS'].unique())


In [ ]:
Inadimplente = df3.loc[df3['STATUS'] == 1].count()[0]
Adimplente = df3.loc[df3['STATUS'] == 0].count()[0]

labels = ['days past due', 'paid off/No loan']
colors = ['#d10000', '#6297e3']
explode = (.1,.1)


plt.pie([Inadimplente, Adimplente], labels = labels, colors = colors, 
        autopct = '%.2f %%', pctdistance= 0.2, startangle=170, explode = explode)
plt.show()

Podemos notar que nosso conjunto de dados está **muito desbalanceado** e a proporção de classes é de 24853 Adimplentes para 281 Inadimplentes. E para isso vamos usar o método de resampling para balancear a base de dados.

In [ ]:
import imblearn
from imblearn.under_sampling import RandomUnderSampler


In [ ]:
x,y = df3.loc[:,df3.columns != 'STATUS'], df3.loc[:,'STATUS']

# Definindo a proporção de dados da classe onde há menos observações
sampling_strategy= 0.34
rus = RandomUnderSampler(sampling_strategy=sampling_strategy)
X_res, y_res = rus.fit_resample(x, y)
autopct = "%.2f"
ax = y_res.value_counts().plot.pie(autopct=autopct)
_ = ax.set_title("Under-sampling")

In [ ]:
# Chamar a variavel STATUS de risco ajuda a entener melhor
colormap = plt.cm.RdBu
plt.figure(figsize=(14,12))
plt.title('Correlation of Features', y=1.05, size=15)
sns.heatmap(df3.astype(float).corr(),linewidths=0.1,vmax=1.0, 
            square=True, cmap=colormap, linecolor='white', annot=True)

As variáveis tem poucas correlação entre si, o que pode ser um sinal bom, diminuindo as chances de inflar o modelo.

## Aplicando a técnica de ensemble stacking

In [ ]:

from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier

In [ ]:
# splitting the data

x_train,x_test,y_train,y_test = train_test_split(X_res,y_res,test_size = 0.20,random_state = 1)

x_train_0,x_train_1,y_train_0,y_train_1 = train_test_split(X_res,y_res,test_size = 0.60, random_state = 1)

In [ ]:
#X_res, y_res

In [ ]:
#criando uma lista com os modelos
# Vendo qual tem a melhor acurácia para usa-lo no stacking

models = {}
models['knn'] = KNeighborsClassifier()
models['cart'] = DecisionTreeClassifier()
models['svm'] = SVC()
models['bayes'] = GaussianNB()
models['rdm'] = RandomForestClassifier()
models['lgc'] = LogisticRegression(max_iter=1000)
models['ada'] = AdaBoostClassifier()
models['gda'] = GradientBoostingClassifier()
models['bca'] = BaggingClassifier()

In [ ]:
# Voting method
# Método de votação
for name, model in models.items():
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    accuracy = accuracy_score(y_test,y_pred)
    print(name, accuracy)
    
    #Suspeita de overfitting analisar pela curva roc

Podemos ver que três modelos estão gerando overfitting, portanto iremos revome-los

In [ ]:
# Rodando novamente
models = {}
models['knn'] = KNeighborsClassifier()
models['svm'] = SVC()
models['bayes'] = GaussianNB()
models['lgc'] = LogisticRegression(max_iter=1000)
models['ada'] = AdaBoostClassifier()
models['gda'] = GradientBoostingClassifier()

# Acurácias

# Voting method
# Método de votação
for name, model in models.items():
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    accuracy = accuracy_score(y_test,y_pred)
    print(name, accuracy)

    
    
# Verificando o intervalo de confiança da acurácia
from sklearn.model_selection import cross_val_score    

knn_IC = cross_val_score(models['knn'], X_res,y_res, cv=5)
svm_IC = cross_val_score(models['svm'], X_res,y_res, cv=5)
bayes_IC = cross_val_score(models['bayes'], X_res,y_res, cv=5)
lgc_IC = cross_val_score(models['lgc'], X_res,y_res, cv=5)
gda_IC = cross_val_score(models['gda'], X_res,y_res, cv=5)
ada_IC = cross_val_score(models['ada'], X_res,y_res, cv=5)

scores = {}

scores['knn'] =  knn_IC.mean() + knn_IC.std() * 2, knn_IC.mean() - knn_IC.std() * 2
scores['svm'] =  svm_IC.mean() + svm_IC.std() * 2, svm_IC.mean() - svm_IC.std() * 2
scores['bayes'] =   bayes_IC.mean() + bayes_IC.std() * 2,bayes_IC.mean() - bayes_IC.std() * 2
scores['lgc'] =  lgc_IC.mean() + lgc_IC.std() * 2, lgc_IC.mean() - lgc_IC.std() * 2
scores['gda'] =  gda_IC.mean() + gda_IC.std() * 2, gda_IC.mean() - gda_IC.std() * 2
scores['ada'] =  ada_IC.mean() + ada_IC.std() * 2, ada_IC.mean() - ada_IC.std() * 2

#Cofidence interval 
scores




In [ ]:

#Avaliando o desempenho dos modelos que tiverem a acurácia dentro do intervalo
# para evitar o paradoxo da Acurácia

from sklearn.metrics import classification_report



models = {}

models['knn'] = KNeighborsClassifier()
models['svm'] = SVC()
models['bayes'] = GaussianNB()
models['lgc'] = LogisticRegression(max_iter=1000)
models['ada'] = AdaBoostClassifier()
models['gda'] = GradientBoostingClassifier()


for name, model in models.items():
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    classification = classification_report(y_test,y_pred)
    print(name, classification)

#### Veja como a acurácia pode enganar a escolha do modelo. Os únicos que teveram a capacidade de classificar os indíviduos como possíveis inadimplentes (Recall), foram o KNN, com 18%, beysiano com 4%, Ada com 11%, e o Gda com 18%. Ou seja, do total de inadimplentes existentes na base proposta, apenas 4 modelos coseguiram fazer essa classificação. Todos os demais conseguiram prever apenas os não inadimplentes, que não é o objetivo de análise desse trabalho.

# Qual a importância de analisar o Recall dos modelos nos estudos de crédito?

####   Quando uma empresa crediticia deseja fornecer crédito aos seus clientes, ela não só analisa as acurácias dos modelos. Na verdade isso componhe a menor parte na análise de crédito. Dado as condições internas da instituição, existe sempre um grau de risco nas aplicações de produtos financeiros, e de acordo com a situação interna da empresa, ela determinará qual individuo receberá seu crédito. Portanto, o ponto que mais afeta quem receberá o crédito, é saber qual é a probabilidade de um cliente com determinadas características vir a se tornar um possível inadimplente, e com isso saber qual é a sua probabilidade de ter atrasos, ou não quitação da dívida, e assim determinar o ponto de corte de acordo com o grau de risco que a instituição escolheu. Por exemplo, o banco X não aumentará o limite de cartão de crédito a clientes que possuem probabilidades maior ou igual a 30% de ser inadimplente. Isso equivale a determinar um ponto de corte de 0,3. Ou seja, nas decisões de quem receberá crédito ou não, não é a acurácia que nos traz o melhor desempenho do modelo, mas sim, o seu desempenho quanto a variações nos pontos de cortes, obtido pela AUC da curva ROC que é traçada a partir do Recall.

#### No nosso caso, o único modelo que teve a mínima capacidade classificar os positivos, foi o Beysiano. Vamos obter a AUC para comparar.

In [ ]:
    from sklearn.metrics import roc_auc_score

    model_bayes = GaussianNB().fit(x_train, y_train)
    model_ada = AdaBoostClassifier().fit(x_train, y_train)
    model_knn = KNeighborsClassifier().fit(x_train, y_train)
    model_gda = GradientBoostingClassifier().fit(x_train, y_train)
    
  
    y_bayes = model_bayes.predict(x_test)
    y_ada = model_ada.predict(x_test)
    y_knn = model_knn.predict(x_test)
    y_gda = model_gda.predict(x_test)
    
    
auc = {}
auc['bayes'] = roc_auc_score(y_test, y_bayes)
auc['ada'] = roc_auc_score(y_test, y_ada)
auc['knn'] = roc_auc_score(y_test, y_knn)
auc['gda'] = roc_auc_score(y_test, y_gda)

auc


In [ ]:
#Agora vamos interpretar nosso modelo com o Lime (ou Shap)